In [ ]:
%matplotlib inline


Autograd: 自动微分
===================================

PyTorch中所有神经网络的核心是`autograd`包。让我们首先简要地看一下，然后我们将去训练我们的第一个神经网络。


autograd软件包为Tensors上的所有操作提供自动微分。它是一个逐个运行的框架，这意味着您的反向传播由您的代码运行方式定义，并且每次迭代都可以不同。


Let us see this in more simple terms with some examples.

Tensor
--------

``torch.Tensor`` 是这个包的核心类。如果你将其
``.requires_grad`` 置为 ``True``， 它就会开始追踪在它之上的所有操作。当你完成计算并调用 ``.backward()`` ，所有的梯度都会自动计算。这个tensor的梯度会被累积到 ``.grad`` 属性.

要阻止Tensor跟踪历史记录，可以调用`.detach()`将其从计算历史记录中分离出来，并防止将来的计算被跟踪。


要防止跟踪历史记录（和使用内存），您还可以使用`torch.no_grad()`包装代码块：在评估模型时，这可能特别有用，因为模型可能具有`requires_grad = True`的可训练参数，但我们不需要梯度。


还有一个类对于autograd实现非常重要 -- ``Function``。


`Tensor`和`Function`互相连接并构建一个非循环图，它编码完整的计算历史。每个张量都有一个`.grad_fn`属性，该属性引用已创建`Tensor`的`Function`（除了用户创建的Tensors--它们的`grad_fn`为`None`）。


如果要计算导数，可以在Tensor上调用`.backward()`。如果Tensor是标量（即它包含一个元素数据），则不需要为`backward()`指定任何参数，但是如果它有更多元素，则需要指定一个`gradient`参数，该参数是形状匹配的张量。




In [2]:
import torch

创建一个张量并设置`requires_grad = True`以跟踪它的计算




In [15]:
x = torch.ones(2, 2, requires_grad=True)
print(x)

tensor([[ 1.,  1.],
        [ 1.,  1.]])


做一个张量的操作：



In [16]:
y = x + 2
print(y)

tensor([[ 3.,  3.],
        [ 3.,  3.]])


`y`是作为操作的结果创建的，因此它具有`grad_fn`。




In [17]:
print(y.grad_fn)

在y上做更多的操作



In [18]:
z = y * y * 3
out = z.mean()

print(z, out)

tensor([[ 27.,  27.],
        [ 27.,  27.]]) tensor(27.)


`.requires_grad_(...)`就地更改现有的`Tensor`的`requires_grad`标志。如果没有给出，输入标志默认为`False`。




In [19]:
a = torch.randn(2, 2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.grad_fn)

False
True


梯度
---------
现在让我们反向传播吧。

因为`out`包含单个标量，所以`out.backward()`等同于`out.backward(torch.tensor(1))`。





In [20]:
out.backward()

输出梯度 d(out)/dx





In [21]:
print(x.grad)

tensor([[ 4.5000,  4.5000],
        [ 4.5000,  4.5000]])


你应该会获得一个 ``4.5``的矩阵。我们把 ``out``
*张量*叫做 “$o$”.
就有 $o = \frac{1}{4}\sum_i z_i$,
$z_i = 3(x_i+2)^2$ 以及 $z_i\bigr\rvert_{x_i=1} = 27$.
因此，
$\frac{\partial o}{\partial x_i} = \frac{3}{2}(x_i+2)$, hence
$\frac{\partial o}{\partial x_i}\bigr\rvert_{x_i=1} = \frac{9}{2} = 4.5$.



你可以利用autograd做很多疯狂的事情。



In [22]:
x = torch.randn(3, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(y)

tensor([ 1337.8494,   165.0922,   607.4825])


In [23]:
gradients = torch.tensor([0.1, 1.0, 0.0001], dtype=torch.float)
y.backward(gradients)

print(x.grad)

tensor([  51.2000,  512.0000,    0.0512])



您也可以通过使用`torch.no_grad()`包装代码块来停止使用`.requires_grad = True`跟踪历史记录上的`autograd`：




In [24]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False


**拓展阅读:**

Documentation of ``autograd`` and ``Function`` is at
http://pytorch.org/docs/autograd

